# Question 1
## Forward Kinimatics


**NumPy is used for numerical array computations and matrix operations. You can do things like:**  
Perform matrix multiplication and other linear algebra operations

```python
import numpy as np

# Define two 4x4 matrices (e.g., transformation matrices in 3D)
A = np.array([
    [1, 0, 0, 1],
    [0, 1, 0, 2],
    [0, 0, 1, 3],
    [0, 0, 0, 1]
])
B = np.array([
    [0, -1, 0, 0],
    [1,  0, 0, 0],
    [0,  0, 1, 0],
    [0,  0, 0, 1]
])

# Multiply the matrices
product = A @ B
print(product)
```

**Sympy is used for symbolic math. You can do things like:**  
Define symbolic variables and build a 4x4 homogeneous transformation matrix for 3D

```python
import sympy as sp

# Define symbolic variables
theta = sp.symbols('theta')
tx, ty, tz = sp.symbols('tx ty tz')

# Build a 4x4 homogeneous transformation matrix for 3D
T = sp.Matrix([
    [sp.cos(theta), -sp.sin(theta), 0, tx],
    [sp.sin(theta),  sp.cos(theta), 0, ty],
    [0,             0,             1, tz],
    [0,             0,             0, 1]
])

# Print Results
sp.pprint(T)
```

**SpatialMath is used for spatial transformations and robotics kinematics in 3D. You can do things like:**  
Create rotation and transformation matrices, and combine rotations with translations<br>
SE and SO stand for Special Euclidian and Orthoganal grouped matrixes


```python
from spatialmath import SE3, SO3
import numpy as np

# Create a 3D rotation matrix: 45° rotation about the z-axis
R = SO3.Rz(np.deg2rad(45))
print("Rotation Matrix:\n", R.A)

# Create a 3D homogeneous transformation: translation + rotation
T = SE3([1, 2, 3]) * SE3.Rz(np.deg2rad(30))
print("Homogeneous Transformation Matrix:\n", T.A)
```

In [3]:
import sympy as sp
import numpy as np
from spatialmath import SO3, SE3

## Current Goal: Find
$^{0}T_{c} = {}^{0}T_{1} \, {}^{1}T_{2} \, {}^{2}T_{c}$

The translation Matrixes are simple

In [108]:
d1, d2, d3 = sp.symbols('d1 d2 d3')

t_0_1 = sp.Matrix([0,0,d1]) #read as the translation of frame 1 in respect to frame 0
t_1_2 = sp.Matrix([0,0,d2])
t_2_c = sp.Matrix([0,0,d3])

The rotation matrixes take a little more effort

In [ ]:
theta1, theta2 = sp.symbols('theta1 theta2')

# Find R_0_1 (read as the rotation matrix of 1 in respect to 0)

Rx90 = SO3.Rx(sp.pi/2) #90 degrees
RyTheta1 = SO3.Ry(theta1)
R_0_1 = Rx90 * RyTheta1

sp.Matrix(R_0_1.A)


Matrix([
[cos(theta1), 0,  sin(theta1)],
[sin(theta1), 0, -cos(theta1)],
[          0, 1,            0]])

In [57]:
# Find R_1_2

RyTheta2 = SO3.Ry(theta2)
R_1_2 = Rx90 * RyTheta2

sp.Matrix(R_1_2.A)

Matrix([
[cos(theta2), 0,  sin(theta2)],
[sin(theta2), 0, -cos(theta2)],
[          0, 1,            0]])

In [ ]:
# Find R_2_c

Rz270 = SO3.Rz(-sp.pi/2)
R_2_c = Rz270

sp.Matrix(R_2_c.A)

Matrix([
[ 0, 1, 0],
[-1, 0, 0],
[ 0, 0, 1]])

So now we can put together the transformation matrixes

In [121]:
def make_transform(R, t):
    """
    Creates a 4x4 homogeneous transformation matrix T from a rotation matrix R and a translation vector t.
    
    Parameters:
      R : A rotation matrix. This can be an object with a .A attribute (like an SO3 object)
          or a standard Sympy Matrix.
      t : A 3x1 translation vector. It should be a column vector.
      
    Returns:
      T : A 4x4 homogeneous transformation matrix as a Sympy Matrix.
    """
    # Convert the rotation matrix to a standard Sympy Matrix if necessary.
    try:
        R_mat = sp.Matrix(R.A)
    except AttributeError:
        R_mat = sp.Matrix(R)
    
    # Ensure t is a column vector.
    t_vec = sp.Matrix(t)
    if t_vec.shape[1] != 1:
        t_vec = t_vec.T if t_vec.shape[0] == 1 else t_vec
    
    # Build the transformation matrix by concatenating R and t, and appending the bottom row.
    T = sp.Matrix.vstack(
        sp.Matrix.hstack(R_mat, t_vec),
        sp.Matrix([[0, 0, 0, 1]])
    )
    return T

In [122]:
T_0_1 = make_transform(R_0_1, t_0_1)
sp.Matrix(T_0_1)

Matrix([
[cos(theta1), 0,  sin(theta1),  0],
[sin(theta1), 0, -cos(theta1),  0],
[          0, 1,            0, d1],
[          0, 0,            0,  1]])

In [123]:
T_1_2 = make_transform(R_1_2, t_1_2)
sp.Matrix(T_1_2)

Matrix([
[cos(theta2), 0,  sin(theta2),  0],
[sin(theta2), 0, -cos(theta2),  0],
[          0, 1,            0, d2],
[          0, 0,            0,  1]])

In [124]:
T_2_c = make_transform(R_2_c, t_2_c)
sp.Matrix(T_2_c)

Matrix([
[ 0, 1, 0,  0],
[-1, 0, 0,  0],
[ 0, 0, 1, d3],
[ 0, 0, 0,  1]])

In [127]:
# Construct final transform matrix

T_0_c = T_0_1 * T_1_2 * T_2_c
sp.Matrix(T_0_c)

Matrix([
[-sin(theta1), cos(theta1)*cos(theta2), sin(theta2)*cos(theta1),  d2*sin(theta1) + d3*sin(theta2)*cos(theta1)],
[ cos(theta1), sin(theta1)*cos(theta2), sin(theta1)*sin(theta2), -d2*cos(theta1) + d3*sin(theta1)*sin(theta2)],
[           0,             sin(theta2),            -cos(theta2),                          d1 - d3*cos(theta2)],
[           0,                       0,                       0,                                            1]])